# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [33]:
import numpy as np
import csv
import pandas as pd
recipes = pd.read_csv('recipes_sample.csv', delimiter = ',')
reviews = pd.read_csv('reviews_sample.csv', delimiter = ',')
recipes.pop('contributor_id')
recipes.pop('n_steps')

0         NaN
1         NaN
2         NaN
3         NaN
4         4.0
         ... 
29995    16.0
29996     NaN
29997     NaN
29998     NaN
29999     9.0
Name: n_steps, Length: 30000, dtype: float64

In [34]:
reviews.head()

,Unnamed: 0,user_id,recipe_id,date,rating,review
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [35]:
reviews_random = reviews.sample(frac = 0.05)
recipes_random = recipes.sample(frac = 0.05)
with pd.ExcelWriter('recipes.xlsx') as writer:
    reviews_random.to_excel(writer, sheet_name = 'Отзывы', index=False)
    recipes_random.to_excel(writer, sheet_name = 'Рецепты', index=False)
print(reviews_random)
print(recipes_random)

        Unnamed: 0  user_id  recipe_id        date  rating  \
40755       766374   378294     193955  2010-04-08       5   
76714       686514   115758     218161  2007-04-11       5   
19007       974404   411567      42169  2008-10-25       5   
68963      1108559    14015      30358  2002-07-17       5   
121781       61339    57695      66352  2005-09-28       5   
...            ...      ...        ...         ...     ...   
31826       442528    79944      98356  2004-09-13       3   
104737      962939   283474     206939  2010-04-13       5   
15375       216760   604017     118236  2008-12-27       5   
36520      1072965   745889      58976  2008-02-03       4   
90324      1065904    55128      13707  2008-01-13       5   

                                                   review  
40755         Couldn't get enough of them!  Great recipe!  
76714   This made for a really tasty dinner. I cut my ...  
19007   Oh WOW, these are good!  No complaints from an...  
68963   I`am sh

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [36]:
import xlwings as xw
wb = xw.Book('recipes.xlsx')
seconds_assign_column = recipes_random[['minutes']] * 60
seconds_assign_column.columns = ['seconds_assign']
sh_arr = wb.sheets['Рецепты']
sh_arr.range('H1').options(index=False).value = seconds_assign_column

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [37]:
fml = xw.Range('I2').formula = f'=D2*60'
xw.Range('I2:I1501').formula = fml
sh_arr.range('I1').value = 'seconds_formula'

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [38]:
sh_arr.range('H1').api.Font.Bold = True
sh_arr.range('I1').api.Font.Bold = True
sh_arr.range('H1').api.HorizontalAlignment = -4108
sh_arr.column_width = 15

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [39]:
for elem in sh_arr.range('C2:C1501'):
    if elem.value < 5:
        elem.color = (0,255,0)
    elif elem.value >= 5 and elem.value <= 10:
        elem.color = (255, 255, 0)
    else:
        elem.color = (255,0,0)

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [47]:
sh_arr.range('J1').value = 'n_reviews'
sh_arr.range('J1').api.Font.Bold = True

count = len(reviews_random)
fml = f'=COUNTIF(Отзывы!$C$2:Отзывы!$C${count},Отзывы!C2)'
sh_arr.range("J2").formula = fml
xw.Range("J2:J1501").formula = fml

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 